### Fine tune InceptionV3 on new set of classes

In [1]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
#from keras import backend as K
from keras.optimizers import Adam,RMSprop
## Let's import all the modules needed to build the neural network
import keras
from keras.models import Sequential,Model
from keras.layers.core import Flatten,Dense,Dropout,Lambda
from keras.layers.convolutional import Convolution2D,MaxPooling2D,ZeroPadding2D
from keras.layers import Input
from keras.optimizers import SGD,RMSprop,Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.data_utils import get_file

Using Theano backend.
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/gmedasani/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [27]:
# Set the path to the image data
#path = "/datascience/datasets/kaggle-cats-vs-dogs/original-kaggle-data/"
#path = "/datascience/datasets/kaggle-cats-vs-dogs/kaggle-data-reorganized/sample/"
path = "/datascience/datasets/kaggle-cats-vs-dogs/kaggle-data-reorganized/"

In [3]:
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

In [4]:
base_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 3, None, None) 0                                            
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 32, None, None 896         input_1[0][0]                    
____________________________________________________________________________________________________
batchnormalization_1 (BatchNorma (None, 32, None, None 128         convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 32, None, None 9248        batchnormalization_1[0][0]       
___________________________________________________________________________________________

In [5]:
len(base_model.layers)

217

In [6]:
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(2, activation='softmax')(x)

In [7]:
# this is the model we will train
model = Model(input=base_model.input, output=predictions)

In [8]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 3, None, None) 0                                            
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 32, None, None 896         input_1[0][0]                    
____________________________________________________________________________________________________
batchnormalization_1 (BatchNorma (None, 32, None, None 128         convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 32, None, None 9248        batchnormalization_1[0][0]       
___________________________________________________________________________________________

In [9]:
len(model.layers)

220

In [10]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

In [11]:
base_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 3, None, None) 0                                            
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 32, None, None 896         input_1[0][0]                    
____________________________________________________________________________________________________
batchnormalization_1 (BatchNorma (None, 32, None, None 128         convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 32, None, None 9248        batchnormalization_1[0][0]       
___________________________________________________________________________________________

In [12]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 3, None, None) 0                                            
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 32, None, None 896         input_1[0][0]                    
____________________________________________________________________________________________________
batchnormalization_1 (BatchNorma (None, 32, None, None 128         convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 32, None, None 9248        batchnormalization_1[0][0]       
___________________________________________________________________________________________

In [13]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [65]:
#Set the variables for training data and validation data generators
batch_size=64
class_mode='categorical'
img_size_expected=(299,299)

In [67]:
#Define a function to get the input images in batches with a size of (224,224) as that is what the VGG model expects
def get_batches(dirname,gen=image.ImageDataGenerator(),class_mode=class_mode,
                target_size=img_size_expected,shuffle=True,batch_size=batch_size):
    print(path+dirname)
    return gen.flow_from_directory(path+dirname,target_size=target_size,class_mode=class_mode,
                           shuffle=shuffle,batch_size=batch_size)

In [68]:
train_batches = get_batches('train',class_mode=class_mode,target_size=img_size_expected,
                            batch_size=batch_size)
valid_batches = get_batches('valid',class_mode=class_mode,target_size=img_size_expected,
                            batch_size=batch_size)

/datascience/datasets/kaggle-cats-vs-dogs/kaggle-data-reorganized/train
Found 22500 images belonging to 2 classes.
/datascience/datasets/kaggle-cats-vs-dogs/kaggle-data-reorganized/valid
Found 2500 images belonging to 2 classes.


In [69]:
nb_epoch=2

In [70]:
# train the model on the new data for a few epochs
#model.fit_generator(...)

## Fit the model on the training batches and report accuracy on the validation sets
model.fit_generator(train_batches, 
                    samples_per_epoch=train_batches.nb_sample, 
                    nb_epoch=nb_epoch,
                    validation_data=valid_batches, 
                    nb_val_samples=valid_batches.nb_sample)

Epoch 1/2
22500/22500 [==============================] - 892s - loss: 0.1014 - acc: 0.9659 - val_loss: 0.0745 - val_acc: 0.9724

In [19]:
# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

(0, 'input_1')
(1, 'convolution2d_1')
(2, 'batchnormalization_1')
(3, 'convolution2d_2')
(4, 'batchnormalization_2')
(5, 'convolution2d_3')
(6, 'batchnormalization_3')
(7, 'maxpooling2d_1')
(8, 'convolution2d_4')
(9, 'batchnormalization_4')
(10, 'convolution2d_5')
(11, 'batchnormalization_5')
(12, 'maxpooling2d_2')
(13, 'convolution2d_9')
(14, 'batchnormalization_9')
(15, 'convolution2d_7')
(16, 'convolution2d_10')
(17, 'batchnormalization_7')
(18, 'batchnormalization_10')
(19, 'averagepooling2d_1')
(20, 'convolution2d_6')
(21, 'convolution2d_8')
(22, 'convolution2d_11')
(23, 'convolution2d_12')
(24, 'batchnormalization_6')
(25, 'batchnormalization_8')
(26, 'batchnormalization_11')
(27, 'batchnormalization_12')
(28, 'mixed0')
(29, 'convolution2d_16')
(30, 'batchnormalization_16')
(31, 'convolution2d_14')
(32, 'convolution2d_17')
(33, 'batchnormalization_14')
(34, 'batchnormalization_17')
(35, 'averagepooling2d_2')
(36, 'convolution2d_13')
(37, 'convolution2d_15')
(38, 'convolution2d_18

In [20]:
# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 172 layers and unfreeze the rest:
for layer in model.layers[:172]:
   layer.trainable = False
for layer in model.layers[172:]:
   layer.trainable = True

In [21]:
# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD,Adam,RMSprop
model.compile(optimizer=RMSprop(lr=0.045, decay=0.9, epsilon=1.0), 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [22]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 3, None, None) 0                                            
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 32, None, None 896         input_1[0][0]                    
____________________________________________________________________________________________________
batchnormalization_1 (BatchNorma (None, 32, None, None 128         convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 32, None, None 9248        batchnormalization_1[0][0]       
___________________________________________________________________________________________

In [ ]:
# Our earlier experiments used momentum [19] with a decay of 0.9, while our 
#best models were achieved using RMSProp [21] with decay
#of 0.9 and  = 1.0. We used a learning rate of 0.045,
#decayed every two epoch using an exponential rate of 0.94.
#In addition, gradient clipping [14] with threshold 2.0 was
#found to be useful to stabilize the training. Model evaluations
#are performed using a running average of the parameters
#computed over time

In [23]:
# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
## Fit the model on the training batches and report accuracy on the validation sets
model.fit_generator(train_batches, 
                    samples_per_epoch=train_batches.nb_sample, 
                    nb_epoch=nb_epoch,
                    validation_data=valid_batches, 
                    nb_val_samples=valid_batches.nb_sample)

Epoch 1/4
250/250 [==============================] - 11s - loss: 9.9114e-04 - acc: 1.0000 - val_loss: 0.1001 - val_acc: 0.9400
Epoch 2/4
250/250 [==============================] - 11s - loss: 0.0014 - acc: 1.0000 - val_loss: 0.0737 - val_acc: 0.9400
Epoch 3/4
250/250 [==============================] - 11s - loss: 0.0026 - acc: 1.0000 - val_loss: 0.0574 - val_acc: 0.9600

In [71]:
#Save the model weights.
model.save_weights('/home/gmedasani/nbs/lesson1-competition/incv3-v1-full-top-2eepochs.h5')

In [92]:
#Load the saved model weights
model.load_weights('/home/gmedasani/nbs/lesson1-competition/incv3-v1-full-top-2eepochs.h5')

In [ ]:
#Run additional eepochs
## Fit the model on the training batches and report accuracy on the validation sets
model.fit_generator(train_batches, 
                    samples_per_epoch=train_batches.nb_sample, 
                    nb_epoch=nb_epoch,
                    validation_data=valid_batches, 
                    nb_val_samples=valid_batches.nb_sample)

Epoch 1/2
 2432/22500 [==>...........................] - ETA: 735s - loss: 0.0918 - acc: 0.9679 

## Make predictions on the test data and write the output as a CSV

In [73]:
batch_size=125
class_mode=None
img_size_expected=(299,299)
test_batches = get_batches('test1',class_mode=class_mode,target_size=img_size_expected,
                            batch_size=batch_size,shuffle=False)

/datascience/datasets/kaggle-cats-vs-dogs/kaggle-data-reorganized/test1
Found 12500 images belonging to 1 classes.


In [74]:
test_batches

In [75]:
test_batches.nb_sample

12500

In [76]:
test_image_ids=test_batches.filenames

In [77]:
import numpy as np
test_image_ids = np.array([int(f.split('/')[1].split('.')[0]) for f in test_image_ids])

In [78]:
test_image_ids

array([   1,   10,  100, ..., 9997, 9998, 9999])

In [79]:
test_preds = model.predict_generator(test_batches,test_batches.nb_sample)

In [80]:
test_preds

array([[  2.71742847e-05,   9.99972820e-01],
       [  9.99998689e-01,   1.35201492e-06],
       [  9.99975920e-01,   2.41389334e-05],
       ..., 
       [  1.77352176e-05,   9.99982238e-01],
       [  9.99981523e-01,   1.85080116e-05],
       [  9.99995232e-01,   4.74337594e-06]], dtype=float32)

In [81]:
test_image_ids.shape

(12500,)

In [82]:
test_preds.shape

(12500, 2)

In [83]:
is_dog_probabilities = test_preds[:,1]
is_dog_probabilities

array([  9.99972820e-01,   1.35201492e-06,   2.41389334e-05, ...,
         9.99982238e-01,   1.85080116e-05,   4.74337594e-06], dtype=float32)

In [84]:
print ("Raw Predictions: " + str(is_dog_probabilities[:5]))
print ("Mid Predictions: " + str(is_dog_probabilities[(is_dog_probabilities < .6) & (is_dog_probabilities > .4)]))
print ("Edge Predictions: " + str(is_dog_probabilities[(is_dog_probabilities == 1) | (is_dog_probabilities == 0)]))

Raw Predictions: [  9.99972820e-01   1.35201492e-06   2.41389334e-05   9.99971271e-01
   9.96965706e-01]
Mid Predictions: [ 0.494362    0.42812967  0.55146641  0.54186827  0.55335581  0.58143812
  0.44944635  0.5628038   0.48883703  0.51434028  0.51120692  0.51216471
  0.57227522  0.55966121  0.5082891   0.44118971  0.57032168  0.48636341
  0.57644451  0.42525211  0.52264535  0.48205012  0.45487937  0.59695715
  0.49371111  0.42329535  0.59820789  0.47294551  0.47639936  0.51325291
  0.49212357  0.45969531  0.41777235  0.5227257   0.4317219   0.45299983
  0.56684488  0.40640891  0.57699054  0.54450411  0.42504925  0.50081825
  0.47188681  0.43212453  0.40988252  0.4682073   0.44317502  0.51839304
  0.56712484  0.52149063  0.56189173  0.43781742  0.59477824  0.54455888
  0.44020239  0.45346785  0.4622269   0.44545445  0.49529028  0.42326435
  0.40240532  0.46896118  0.53182989  0.44238624  0.59722805  0.41266015
  0.54840755  0.58829516  0.59435076  0.55239898  0.43954968  0.4215568
  0

In [85]:
#So to play it safe, we use a sneaky trick to round down our edge predictions
#Swap all ones with .95 and all zeros with .05
is_dog_probabilities = is_dog_probabilities.clip(min=0.01, max=0.99)

In [86]:
print ("Raw Predictions: " + str(is_dog_probabilities[:5]))
print ("Mid Predictions: " + str(is_dog_probabilities[(is_dog_probabilities < .6) & (is_dog_probabilities > .4)]))
print ("Edge Predictions: " + str(is_dog_probabilities[(is_dog_probabilities == 1) | (is_dog_probabilities == 0)]))

Raw Predictions: [ 0.99000001  0.01        0.01        0.99000001  0.99000001]
Mid Predictions: [ 0.494362    0.42812967  0.55146641  0.54186827  0.55335581  0.58143812
  0.44944635  0.5628038   0.48883703  0.51434028  0.51120692  0.51216471
  0.57227522  0.55966121  0.5082891   0.44118971  0.57032168  0.48636341
  0.57644451  0.42525211  0.52264535  0.48205012  0.45487937  0.59695715
  0.49371111  0.42329535  0.59820789  0.47294551  0.47639936  0.51325291
  0.49212357  0.45969531  0.41777235  0.5227257   0.4317219   0.45299983
  0.56684488  0.40640891  0.57699054  0.54450411  0.42504925  0.50081825
  0.47188681  0.43212453  0.40988252  0.4682073   0.44317502  0.51839304
  0.56712484  0.52149063  0.56189173  0.43781742  0.59477824  0.54455888
  0.44020239  0.45346785  0.4622269   0.44545445  0.49529028  0.42326435
  0.40240532  0.46896118  0.53182989  0.44238624  0.59722805  0.41266015
  0.54840755  0.58829516  0.59435076  0.55239898  0.43954968  0.4215568
  0.56253552  0.52048051  0.5

In [87]:
#Combine the is_dog_probabilities and image ids
test_id_labels = np.stack([test_image_ids,is_dog_probabilities], axis=1)

In [88]:
test_id_labels.shape

(12500, 2)

In [90]:
test_id_labels

array([[  1.00000000e+00,   9.90000010e-01],
       [  1.00000000e+01,   9.99999978e-03],
       [  1.00000000e+02,   9.99999978e-03],
       ..., 
       [  9.99700000e+03,   9.90000010e-01],
       [  9.99800000e+03,   9.99999978e-03],
       [  9.99900000e+03,   9.99999978e-03]])

In [91]:
submission_file_name = 'incv3-submission3.csv'
np.savetxt(submission_file_name, test_id_labels, fmt='%d,%.5f', header='id,label', comments='')